In [ ]:
#import stat_lib as SL
import numpy as np
import pandas as pd
from astropy.table import Table, Column, vstack
import inspect
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import warnings
import scipy.stats as stats
from scipy.stats import ks_2samp

In [ ]:
sfcg_235 = Table.read('/home/seba/Documents/DECALS/Galaxies/Galaxies_DECALS_235.csv', format='csv')
sfcg_235.add_column(Column([5000] * len(sfcg_235), name='radvel'))
sfcg_235.rename_column('Gal', 'GAL')
sfcg_235['GAL'] = [str(valor) for valor in sfcg_235['GAL']]

In [ ]:
tabla = Table.read('/home/seba/Documents/MorphoLS/Catalog/Galaxies_DECALS_sex_final.csv')
tabla.add_column(Column([5000] * len(tabla), name='radvel'))

tabla.add_column(Column(data=range(1, len(tabla) + 1), name='GAL'))

tabla['GAL'] = [str(valor) for valor in tabla['GAL']]
grouped = tabla.group_by('Group')
GL = grouped.groups.keys

In [ ]:
tabla_cs = Table.read('/home/seba/Documents/MorphoLS/Analysis/Control_Sample_1144.csv')
tabla_cs.add_column(Column([5000] * len(tabla_cs), name='radvel'))
tabla_cs.rename_column('index', 'GAL')
tabla_cs['GAL'] = [str(valor) for valor in tabla_cs['GAL']]

In [ ]:
tabla_ordenada = tabla[np.argsort(tabla['Group'])]
tabla_ordenada

In [ ]:
codigo_funcion = inspect.getfile(SL.ex_phot_mod_decals)
print(codigo_funcion)

In [ ]:
output=SL.ex_phot_mod_decals(sfcg_235['GAL'][0], sfcg_235, size_image_phy=30, snr = 2.5, 
                             area_min=0.5, sky_box = [15,15], area_min_deblend=1,deblend='off',
                             band='r', roundhi=0.5, roundlo=-0.5, run_auto='yes', threshold=5,aper_center=50,
                             show_plot='yes', plot_model='no', mask_stars='yes')

In [ ]:
gal_norun = []
for i, gal in enumerate(sfcg_235['GAL'][0:len(sfcg_235)]):
    print ('\n##############')
    print (i, gal)
    try:
        output=SL.ex_phot_mod_decals(gal, sfcg_235, size_image_phy=30, band='g', 
                                      snr = 2.5, area_min=5, deblend='on', area_min_deblend=5, 
                                      sky_box = [5,5], 
                                      mask_stars='yes', roundhi=0.5, roundlo=-0.5, 
                                      run_auto='yes', show_plot='no', outputfile = 'properties_try.csv')
    except:
        print ('something was wrong')
        gal_norun.append(gal)
t = Table.read('properties_try.csv', format='ascii')
t.add_column(Column([235]*len(sfcg_235), name='Group'))
t.write('properties_try.csv', overwrite=True)

In [ ]:
prop = Table.read('properties_try.csv', format='ascii')
prop

In [ ]:
def no_param(group):
    gal_norun = []
    tab = tabla[tabla['Group'] == group]
    for i, gal in enumerate(tab['GAL'][0:len(tab)]):
        print ('\n##############')
        print (i, gal)
        try:
            output=SL.ex_phot_mod_decals(gal, tab, size_image_phy=30, band='r', 
                                      snr = 2.5, area_min=5, deblend='on', area_min_deblend=5, 
                                      sky_box = [5,5], 
                                      mask_stars='yes', roundhi=0.5, roundlo=-0.5, 
                                      run_auto='yes', show_plot='yes', outputfile=f'properties_group_{group}.csv')        
        except:
            print ('something was wrong')
            gal_norun.append(gal)
    t = Table.read(f'properties_group_{group}.csv', format='ascii')
    t.add_column(Column([f'{group}']*len(t), name='Group'))
    t.write(f'properties_group_{group}.csv', overwrite=True)

In [ ]:
gal_norun = []
tab = tabla_cs
for i, gal in enumerate(tab['GAL'][0:len(tab)]):
    print ('\n##############')
    print (i, gal)
    try:
        output=SL.ex_phot_mod_decals(gal, tab, size_image_phy=30, band='r', 
                                    snr = 2.5, area_min=5, deblend='on', area_min_deblend=5, 
                                    sky_box = [5,5], 
                                    mask_stars='yes', roundhi=0.5, roundlo=-0.5, 
                                    run_auto='yes', show_plot='yes', outputfile=f'properties_gal_{gal}.csv')
        t = Table.read(f'properties_gal_{gal}.csv', format='ascii')
        t.add_column(Column([f'{gal}']*len(t), name='index'))
        t.write(f'properties_gal_{gal}.csv', overwrite=True)
    except:
        print ('something was wrong')
        gal_norun.append(gal)


In [ ]:
for g in GL['Group']:
    no_param(g)

In [ ]:
properties = Table.read('DECALS_Astromorphlib_xmatch_all.csv', format='ascii')
properties = properties[properties['type'] != 'PSF']
#properties_df = properties.to_pandas()
properties = properties[(properties['Gini'] != 0.0) & (properties['M20'] != 0.0)]
properties = properties[properties['A'] != 100]

In [ ]:
properties_cs = Table.read('DECALS_Astromorphlib_control_sample_xmatch.csv', format='ascii')
properties_cs = properties_cs[properties_cs['type'] != 'PSF']
#properties_df = properties.to_pandas()
properties_cs = properties_cs[(properties_cs['Gini'] != 0.0) & (properties_cs['M20'] != 0.0)]
properties_cs = properties_cs[properties_cs['A'] != 100]

In [ ]:
print("Rango de Gini:", properties['Gini'].min(), "-", properties['Gini'].max())
print("Rango de M20:", properties['M20'].min(), "-", properties['M20'].max())

In [ ]:
from matplotlib.patches import Patch

ginis = properties['Gini']
m20s = properties['M20']
print(len(ginis), len(m20s))

# Definimos las ecuaciones de las líneas
def line_1(x):
    return -0.14 * x + 0.33

def line_2(x):
    return 0.14 * x + 0.80

# Clasificación basada en la posición relativa
colors = []
mergers_g = []
early_g = []
late_g = []
for gini, m20 in zip(ginis, m20s):
    if gini < line_1(m20) and gini > line_2(m20):  # Encima de ambas líneas (Mergers)
        colors.append('indianred')
        early_g.append(gini)
    elif gini <= line_1(m20) and gini < line_2(m20):  # Debajo de la línea roja (Sb/Sc/Ir)
        colors.append('cornflowerblue')
        late_g.append(gini)
    else:  # Entre las líneas roja y verde (E/S0/Sa)
        colors.append('darkgreen')
        mergers_g.append(gini)

# Generar las líneas y el gráfico
lim_2 = (0.8 - 0.33) / (-0.14 - 0.14)
x_line_1 = np.linspace(-3, 0, 100)
x_line_2 = np.linspace(-3, lim_2, 100)
y_line_1 = line_1(x_line_1)
y_line_2 = line_2(x_line_2)

plt.figure(figsize=(9, 9))

plt.plot(x_line_1, y_line_1, color='seagreen')
plt.plot(x_line_2, y_line_2, color='indianred')
sns.kdeplot(x=m20s, y=ginis, cmap='Blues', alpha=0.6, fill=True)

# Colorear los puntos según su clasificación
sns.scatterplot(x=m20s, y=ginis, c=colors, s=15)

# Crear leyenda personalizada
legend_elements = [
    Patch(facecolor='indianred', label='E/S0/Sa'),
    Patch(facecolor='cornflowerblue', label='Sb/Sc/Ir'),
    Patch(facecolor='seagreen', label='Mergers')
]
plt.legend(handles=legend_elements, fontsize=13, loc='upper left')

plt.xlim(0, -2.7)
plt.ylim(0.35, 0.8)
plt.xlabel(r'$M_{20}$')
plt.ylabel(r'$G$')
#plt.title('Gini - M20 Classification System', fontsize=14)
#plt.legend(fontsize=14)

plt.text(-2.2, 0.5, 'E/S0/Sa', fontsize=16, color='indianred')
plt.text(-1.8, 0.40, 'Sb/Sc/Ir', fontsize=16, color='cornflowerblue')
plt.text(-0.7, 0.6, 'Mergers', fontsize=16, color='darkgreen')

plt.show()

In [ ]:
print(f'La cantidad de galaxias Late-Type es: {len(late_g)}, que corresponde a un {round((len(late_g)/len(properties))*100, 2)}%')

print(f'La cantidad de galaxias Early-Type es: {len(early_g)}, que corresponde a un {round((len(early_g)/len(properties))*100, 2)}%')

print(f'La cantidad de galaxias en Mergers es: {len(mergers_g)}, que corresponde a un {round((len(mergers_g)/len(properties))*100, 2)}%')

In [ ]:
# Clasificación y creación de la columna 'tipo'
tipo = []
for gini, m20 in zip(properties['Gini'], properties['M20']):
    if gini < line_1(m20) and gini > line_2(m20): 
        tipo.append('Early')
    elif gini <= line_1(m20) and gini < line_2(m20): 
        tipo.append('Late')
    else:  
        tipo.append('Merger')

# Agregar la columna 'tipo' a la tabla original
properties['Tipo'] = Column(tipo, dtype='str')

# Mostrar la tabla para verificar
print(len(properties[properties['Tipo'] == 'Merger']))

In [ ]:
# Clasificación y creación de la columna 'tipo'
tipo = []
for gini, m20 in zip(properties_cs['Gini'], properties_cs['M20']):
    if gini < line_1(m20) and gini > line_2(m20): 
        tipo.append('Early')
    elif gini <= line_1(m20) and gini < line_2(m20): 
        tipo.append('Late')
    else:  
        tipo.append('Merger')

# Agregar la columna 'tipo' a la tabla original
properties_cs['Tipo'] = Column(tipo, dtype='str')

# Mostrar la tabla para verificar
print(properties_cs)

In [ ]:
print((properties[properties['Tipo'] == 'Merger']))

In [ ]:
ginis = properties['Gini']
m20s = properties['M20']
rgroups = properties['r_theta']  # Asegúrate de que esta columna exista en 'properties'

print(len(ginis), len(m20s), len(rgroups))

# Definimos las ecuaciones de las líneas
def line_1(x):
    return -0.14 * x + 0.33

def line_2(x):
    return 0.14 * x + 0.80

# Generar las líneas
lim_2 = (0.8 - 0.33) / (-0.14 - 0.14)
x_line_1 = np.linspace(-2.6, 0, 100)
x_line_2 = np.linspace(-2.6, lim_2, 100)
y_line_1 = line_1(x_line_1)
y_line_2 = line_2(x_line_2)

# Crear un mapa de colores normalizado para la barra de color
norm = mcolors.Normalize(vmin=min(rgroups), vmax=max(rgroups))
cmap = cm.plasma  # Puedes cambiar el mapa de colores según prefieras

plt.figure(figsize=(12, 10))

# Dibujar las líneas
plt.plot(x_line_1, y_line_1, color='seagreen', label='Merger division')
plt.plot(x_line_2, y_line_2, color='indianred', label='Early/Late Type division')

# Colorear los puntos según los valores de rgroup
sc = plt.scatter(m20s, ginis, c=rgroups, cmap=cmap, s=8)

# Configurar los límites y etiquetas
plt.xlim(0, -2.6)
plt.ylim(0.3, 0.75)
plt.xlabel(r'$M_{20}$', fontsize=14)
plt.ylabel(r'$G$', fontsize=14)
plt.title('Gini - M20 Classification System', fontsize=16)

# Etiquetas para las regiones
plt.text(-2.2, 0.5, 'E/S0/Sa', fontsize=16, color='indianred')
plt.text(-1.8, 0.40, 'Sb/Sc/Ir', fontsize=16, color='cornflowerblue')
plt.text(-0.7, 0.6, 'Mergers', fontsize=16, color='seagreen')

# Añadir la barra de color
cbar = plt.colorbar(sc)
cbar.set_label('Group Size (arcmin)', fontsize=14)

plt.legend(fontsize=14)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Variables iniciales
ginis = properties['Gini']
m20s = properties['M20']
rgroups = properties['r_theta']  # Asegúrate de que esta columna exista en 'properties'

print(len(ginis), len(m20s), len(rgroups))

# Definimos las ecuaciones de las líneas
def line_1(x):
    return -0.14 * x + 0.33

def line_2(x):
    return 0.14 * x + 0.80

# Generar las líneas
lim_2 = (0.8 - 0.33) / (-0.14 - 0.14)
x_line_1 = np.linspace(-3, 0, 100)
x_line_2 = np.linspace(-3, lim_2, 100)
y_line_1 = line_1(x_line_1)
y_line_2 = line_2(x_line_2)

# Crear un mapa de colores normalizado para la barra de color
norm = mcolors.Normalize(vmin=min(rgroups), vmax=max(rgroups))
cmap = cm.PRGn  # Puedes cambiar el mapa de colores según prefieras

plt.figure(figsize=(12, 10))

# Dibujar las líneas
plt.plot(x_line_1, y_line_1, color='seagreen', label='Merger division')
plt.plot(x_line_2, y_line_2, color='indianred', label='Early/Late Type division')

# Crear un hexbin plot
hb = plt.hexbin(m20s, ginis, C=rgroups, gridsize=(17, 20), cmap=cmap, reduce_C_function=np.mean, mincnt=1, norm=norm)

# Configurar los límites y etiquetas
plt.xlim(0, -2.7)
plt.ylim(0.32, 0.8)
plt.xlabel(r'$M_{20}$', fontsize=14)
plt.ylabel(r'$G$', fontsize=14)
plt.title('Gini - M20 Classification System', fontsize=16)

# Etiquetas para las regiones
plt.text(-2.2, 0.5, 'E/S0/Sa', fontsize=16, color='indianred')
plt.text(-1.8, 0.40, 'Sb/Sc/Ir', fontsize=16, color='cornflowerblue')
plt.text(-0.6, 0.6, 'Mergers', fontsize=16, color='seagreen')

# Añadir la barra de color
cbar = plt.colorbar(hb)
cbar.set_label('Group Size (arcmin)', fontsize=14)

# Añadir la leyenda
plt.legend(fontsize=14)
plt.show()


In [ ]:
def filtrado(archivo):
    grouped = archivo.group_by('Group')

    valid_groups=[]

    for group in grouped.groups:
        if any (group['zmemb'] != 0):
            valid_groups.append(group['Group'][0])
    print(len(valid_groups))
    filtered_table = archivo[np.isin(archivo['Group'], valid_groups)]

    # Crear una función para reemplazar los valores de 'zmemb' que son 0
    def replace_zeros(group):
        non_zero_values = group['zmemb'][group['zmemb'] != 0]
        if len(non_zero_values) > 0:
            replacement_value = non_zero_values[0]
            group['zmemb'] = np.where(group['zmemb'] == 0, replacement_value, group['zmemb'])
        return group

    # Aplicar la función a cada grupo y unir los resultados
    result_groups = []
    for group in filtered_table.group_by('Group').groups:
        result_groups.append(replace_zeros(group))

    final_table = vstack(result_groups)
    
    return final_table

In [ ]:
properties_z = filtrado(properties)
H_0 = 67.4 #km/s/Mpc
c = 300000
properties_z['Distance (Kpc)'] = ((properties_z['zUV']*c)/H_0)*1000
properties_z['group_size_kpc'] = properties_z['Distance (Kpc)']*(properties_z['r_theta']*2.9e-4)
properties_z = properties_z[~properties_z['group_size_kpc'].mask]
properties_z

In [ ]:
# Variables iniciales
ginis = properties_z['Gini']
m20s = properties_z['M20']
rgroups = properties_z['group_size_kpc']  # Asegúrate de que esta columna exista en 'properties'

print(len(ginis), len(m20s), len(rgroups))

# Definimos las ecuaciones de las líneas
def line_1(x):
    return -0.14 * x + 0.33

def line_2(x):
    return 0.14 * x + 0.80

# Generar las líneas
lim_2 = (0.8 - 0.33) / (-0.14 - 0.14)
x_line_1 = np.linspace(-3, 0, 100)
x_line_2 = np.linspace(-3, lim_2, 100)
y_line_1 = line_1(x_line_1)
y_line_2 = line_2(x_line_2)

# Crear un mapa de colores normalizado para la barra de color
norm = mcolors.Normalize(vmin=min(rgroups), vmax=max(rgroups))
cmap = cm.PRGn  # Puedes cambiar el mapa de colores según prefieras

plt.figure(figsize=(6, 5))

# Dibujar las líneas
plt.plot(x_line_1, y_line_1, color='seagreen', label='Merger division')
plt.plot(x_line_2, y_line_2, color='indianred', label='Early/Late Type division')

# Crear un hexbin plot
hb = plt.hexbin(m20s, ginis, C=rgroups, gridsize=(17, 20), cmap=cmap, reduce_C_function=np.mean, mincnt=1, norm=norm)

# Configurar los límites y etiquetas
plt.xlim(0, -2.7)
plt.ylim(0.35, 0.8)
plt.xlabel(r'$M_{20}$')
plt.ylabel(r'$G$')
#plt.title('Gini - M20 Classification System', fontsize=16)

# Etiquetas para las regiones
plt.text(-2.2, 0.5, 'E/S0/Sa', fontsize=9, color='indianred')
plt.text(-1.8, 0.40, 'Sb/Sc/Ir', fontsize=9, color='cornflowerblue')
plt.text(-0.6, 0.6, 'Mergers', fontsize=9, color='seagreen')

# Añadir la barra de color
cbar = plt.colorbar(hb)
cbar.set_label('Group Size (Kpc)', fontsize=9)

# Añadir la leyenda
plt.legend(fontsize=9, loc='lower left')
plt.show()

In [ ]:
def bootstrap_median_error(data, n_bootstrap=1000, ci=68):
    """
    Calcula el error de la mediana usando bootstrapping.

    Parámetros:
    - data: array-like, los datos de entrada.
    - n_bootstrap: int, número de resamples (1000 por defecto).
    - ci: int, intervalo de confianza deseado (en porcentaje, 68 o 95 por ejemplo).

    Retorna:
    - mediana: la mediana calculada de los datos originales.
    - error_low: desviación negativa respecto a la mediana.
    - error_high: desviación positiva respecto a la mediana.
    """
    medians = []
    for _ in range(n_bootstrap):
        resample = np.random.choice(data, size=len(data), replace=True)
        medians.append(np.median(resample))
    
    medians = np.array(medians)
    median = np.median(data)
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    error_low = median - np.percentile(medians, lower_percentile)
    error_high = np.percentile(medians, upper_percentile) - median
    
    return median, error_low, error_high
# Definimos las ecuaciones de las líneas
def line_1(x):
    return -0.14 * x + 0.33

def line_2(x):
    return 0.14 * x + 0.80

In [ ]:
# Datos
size_mergers = properties_z['group_size_kpc'][properties_z['Tipo'] == 'Merger']
size_late = properties_z['group_size_kpc'][properties_z['Tipo'] == 'Late']
size_early = properties_z['group_size_kpc'][properties_z['Tipo'] == 'Early']
# Crear bins
bin_edges = np.linspace(min(size_mergers.min(), size_late.min()), 
                        max(size_mergers.max(), size_late.max()), 
                        21)

# Ejemplo de uso para tus distribuciones
median_mergers, err_low_mergers, err_high_mergers = bootstrap_median_error(size_mergers)
median_late, err_low_late, err_high_late = bootstrap_median_error(size_late)
median_early, err_low_early, err_high_early = bootstrap_median_error(size_early)

# Imprimir resultados
print(f"Mergers Median: {median_mergers:.2f} (+{err_high_mergers:.2f} / -{err_low_mergers:.2f})")
print(f"Late Median: {median_late:.2f} (+{err_high_late:.2f} / -{err_low_late:.2f})")
print(f"Early Median: {median_early:.2f} (+{err_high_early:.2f} / -{err_low_early:.2f})")


# Crear la figura
plt.figure(figsize=(5, 5))

# Histogramas
sns.histplot(size_mergers, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='seagreen', label='Mergers', edgecolor='black', linewidth=0.3)
sns.histplot(size_late, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='cornflowerblue', label='Late', edgecolor='black', linewidth=0.3)
sns.histplot(size_early, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='indianred', label='Early', edgecolor='black', linewidth=0.3)

plt.axvline(median_mergers, color='darkgreen', linestyle='--', linewidth=3, label='Median Mergers')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers - err_low_mergers, x2=median_mergers + err_high_mergers, 
                  color='darkgreen', alpha=0.4)
plt.axvline(median_late, color='cornflowerblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late - err_low_late, x2=median_late + err_high_late, 
                  color='cornflowerblue', alpha=0.4)
plt.axvline(median_early, color='indianred', linestyle='--', linewidth=3, label='Median Early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early - err_low_early, x2=median_early + err_high_early, 
                  color='indianred', alpha=0.4)

# Etiquetas y título
plt.xlabel('Group Size (kpc)', fontsize=9)
plt.ylabel('Fraction of galaxies', fontsize=9)
plt.xticks(size=9)
plt.yticks(size=9)

# Leyenda
plt.legend(fontsize=7)

# Mostrar gráfico
plt.show()

print(f"Median Difference (Late - Mergers): {median_late - median_mergers}")

In [ ]:
ginis_cs = properties_cs['Gini']
m20s_cs = properties_cs['M20']
print(len(ginis_cs), len(m20s_cs))


# Clasificación basada en la posición relativa
colors = []
mergers = []
early = []
late = []
for gini, m20 in zip(ginis_cs, m20s_cs):
    if gini < line_1(m20) and gini > line_2(m20):  
        colors.append('indianred')
        early.append(gini)
    elif gini <= line_1(m20) and gini < line_2(m20):  
        colors.append('cornflowerblue')
        late.append(gini)
    else:  
        colors.append('darkgreen')
        mergers.append(gini)

# Generar las líneas y el gráfico
lim_2 = (0.8 - 0.33) / (-0.14 - 0.14)
x_line_1 = np.linspace(-3, 0, 100)
x_line_2 = np.linspace(-3, lim_2, 100)
y_line_1 = line_1(x_line_1)
y_line_2 = line_2(x_line_2)

plt.figure(figsize=(9, 9))

plt.plot(x_line_1, y_line_1, color='seagreen')
plt.plot(x_line_2, y_line_2, color='indianred')
sns.kdeplot(x=m20s_cs, y=ginis_cs, cmap='Oranges', alpha = 0.6, fill=True)

# Colorear los puntos según su clasificación
sns.scatterplot(x=m20s_cs, y=ginis_cs, c=colors, label='Field Galaxies', s=15)

# Crear leyenda personalizada
legend_elements = [
    Patch(facecolor='indianred', label='E/S0/Sa'),
    Patch(facecolor='cornflowerblue', label='Sb/Sc/Ir'),
    Patch(facecolor='seagreen', label='Mergers')
]
plt.legend(handles=legend_elements, fontsize=13, loc='upper left')

plt.xlim(0, -2.7)
plt.ylim(0.35, 0.8)
plt.xlabel(r'$M_{20}$')
plt.ylabel(r'$G$')

plt.text(-2.2, 0.5, 'E/S0/Sa', fontsize=16, color='indianred')
plt.text(-1.8, 0.40, 'Sb/Sc/Ir', fontsize=16, color='cornflowerblue')
plt.text(-0.7, 0.6, 'Mergers', fontsize=16, color='darkgreen')

plt.show()

In [ ]:
print(f'La cantidad de galaxias Late-Type es: {len(late)}, que corresponde a un {round((len(late)/len(properties_cs))*100, 2)}%')

print(f'La cantidad de galaxias Early-Type es: {len(early)}, que corresponde a un {round((len(early)/len(properties_cs))*100, 2)}%')

print(f'La cantidad de galaxias en Mergers es: {len(mergers)}, que corresponde a un {round((len(mergers)/len(properties_cs))*100, 2)}%')

In [ ]:
early_type_sfcg = properties[properties['Tipo'] == 'Early']
late_type_sfcg = properties[properties['Tipo'] == 'Late']
merger_type_sfcg = properties[properties['Tipo'] == 'Merger']

early_type_cs = properties_cs[properties_cs['Tipo'] == 'Early']
late_type_cs = properties_cs[properties_cs['Tipo'] == 'Late']
merger_type_cs = properties_cs[properties_cs['Tipo'] == 'Merger']

In [ ]:
tabla_names = ['E/S0/Sa', 'Sb/Sc/Ir', 'Mergers']

tablas_sfcg = [early_type_sfcg, late_type_sfcg, merger_type_sfcg]
tablas_cs = [early_type_cs, late_type_cs, merger_type_cs]

# Crear una lista para almacenar las fracciones para cada tabla
fractions = []
amount = []
for tabla in tablas_sfcg:
    sf_galaxies = len(tabla)
    amount.append([sf_galaxies])
    # Calcular la fracción respecto al total
    sf_fraction = sf_galaxies / len(properties)
    
    # Añadir las fracciones a la lista
    fractions.append([sf_fraction])

fractions_cs = []
amount_cs = [] 
for tabla_cs in tablas_cs:
    field_galaxies = len(tabla_cs)
    amount_cs.append([field_galaxies])
    field_fraction = field_galaxies / len(properties_cs)
    
    # Añadir las fracciones a la lista
    fractions_cs.append([field_fraction])

# Convertir los resultados en un array de numpy
fractions = np.array(fractions)
fractions_cs = np.array(fractions_cs)
amount = np.array(amount)
amount_cs = np.array(amount_cs)
print(fractions*100, fractions_cs*100)
print(amount, amount_cs)
# Crear el histograma apilado
fig, ax = plt.subplots(figsize=(10, 6))

# Definir las posiciones de los bins (uno para cada tabla)
bins = np.arange(len(tabla_names))

bar1 = ax.bar(bins, fractions[:, 0], alpha = 0.8, label='SFCG Galaxies', color='darkviolet')
bar2 = ax.bar(bins, fractions_cs[:, 0], alpha = 0.5, label='Field Galaxies', color='teal')

ax.set_xticks(bins)
ax.set_xticklabels(tabla_names)
#ax.set_xlabel('Galaxy Type', fontsize=16)
ax.set_ylabel('Fraction of Galaxies', fontsize=16)
#ax.set_title('Fraction of Galaxy Types', fontsize=16)
ax.legend(fontsize=14)

plt.show()

In [ ]:
# Lista de parámetros para graficar
parameters = ["F(G_M20)", "C", "A", "Gini", "M20"]
# Definir límites personalizados para cada parámetro
x_limits = {
    "F(G_M20)": [-1.5, 1.5],
    "C": [2.0, 5],
    "A": [-0.5, 0.8],
    "Gini": [0.3, 0.8],
    "M20": [-2.5, -0.5]
}
# Crear subplots con dos filas (una para cada dataset) y múltiples columnas
n_params = len(parameters)
fig, axes = plt.subplots(2, n_params, figsize=(30, 15), sharey='row', sharex='col')

for i, param in enumerate(parameters):
    # Fila 0: Dataset properties
    sns.kdeplot(properties[param], ax=axes[0, i], color="darkviolet", alpha=0.5, fill = True, label="Properties", common_norm=True)
    median_properties = np.median(properties[param])
    axes[0, i].axvline(median_properties, color="darkviolet", linestyle="--", linewidth=5, label="Mediana")
    axes[0, i].set_title(param.replace("_", " ").capitalize(), fontsize=25)
    axes[0, i].set_ylabel("Densidad")
    axes[0, i].legend(fontsize=20)
    
    # Aplicar límites del eje x
    if param in x_limits:
        axes[0, i].set_xlim(x_limits[param])

    # Fila 1: Dataset properties_cs
    sns.kdeplot(properties_cs[param], ax=axes[1, i], color="teal", alpha=0.5, fill=True, label="Properties CS", common_norm=True)
    median_properties_cs = np.median(properties_cs[param])
    axes[1, i].axvline(median_properties_cs, color="teal", linestyle="--", linewidth=5, label="Mediana")
    axes[1, i].set_ylabel("Densidad", fontsize=25)
    axes[1, i].legend(fontsize=20)
    
    if param in x_limits:
        axes[1, i].set_xlim(x_limits[param])

# Ajustar diseño de subplots
plt.xticks(size=20)
plt.yticks(size=20)
plt.tight_layout()
plt.show()

In [ ]:
decals_raw = Table.read('amlb_decals_for_regions.csv', format='ascii')
decals = decals_raw[decals_raw['type'] != 'PSF']
filtros = ['g', 'r', 'i', 'z']
#filtros_splus = ['G', 'R', 'I', 'Z']
for filtro in filtros:        
    decals['n_' + filtro] = pd.to_numeric(decals['n_' + filtro], errors='coerce')
    decals['RE_' +filtro] = pd.to_numeric(decals['RE_' + filtro], errors = 'coerce')
    decals['e_n_' + filtro] = pd.to_numeric(decals['e_n_' + filtro], errors='coerce')
    decals['e_RE_' +filtro] = pd.to_numeric(decals['e_RE_' + filtro], errors = 'coerce')

# Clasificación y creación de la columna 'tipo'
tipo = []
for gini, m20 in zip(decals['Gini'], decals['M20']):
    if gini < line_1(m20) and gini > line_2(m20): 
        tipo.append('Early')
    elif gini <= line_1(m20) and gini < line_2(m20): 
        tipo.append('Late')
    else:  
        tipo.append('Merger')

# Agregar la columna 'tipo' a la tabla original
decals['Tipo'] = Column(tipo, dtype='str')

In [ ]:
sfr_decals = Table.read('sfr_decals_k_corrected_asmlb.csv', format='csv')

# Clasificación y creación de la columna 'tipo'
tipo = []
for gini, m20 in zip(sfr_decals['Gini'], sfr_decals['M20']):
    if gini < line_1(m20) and gini > line_2(m20): 
        tipo.append('Early')
    elif gini <= line_1(m20) and gini < line_2(m20): 
        tipo.append('Late')
    else:  
        tipo.append('Merger')

# Agregar la columna 'tipo' a la tabla original
sfr_decals['Tipo'] = Column(tipo, dtype='str')

In [ ]:
decals_no_zero = sfr_decals[sfr_decals['n_r']!=0.0]
g_r_decals = decals_no_zero['m_g_corrected'] - decals_no_zero['m_r_corrected']
n_r = decals_no_zero['n_r']
ginis = decals_no_zero['Gini']
m20s = decals_no_zero['M20']

mask_merger = decals_no_zero['Tipo'] == 'Merger'
mask_late = decals_no_zero['Tipo'] == 'Late'
mask_early = decals_no_zero['Tipo'] == 'Early'


# Crear el gráfico de (g-r) vs. n(r)
plt.figure(figsize=(8, 8))

# Fondo KDEplot del diagrama de densidad
#sns.kdeplot(x=n_r, y=g_r_decals, cmap='Blues', alpha=0.6, fill=True, cbar=True)

# Añadir galaxias clasificadas como late, merger y early
sns.scatterplot(x=decals_no_zero['n_r'][mask_late], y=g_r_decals[mask_late], c='cornflowerblue', label='Late-Type', s=15)
sns.scatterplot(x=decals_no_zero['n_r'][mask_merger], y=g_r_decals[mask_merger], c='darkgreen', label='Mergers', s=15)
sns.scatterplot(x=decals_no_zero['n_r'][mask_early], y=g_r_decals[mask_early], c='indianred', label='Early-Type', s=15)

# Personalización del gráfico
plt.ylim(-0.4, 1.6)
plt.xlim(-1, 10)  # Ajustar según los datos reales de n(r)
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel('n(r)', size=15)
plt.ylabel('$(g-r)_{0}$', size=15)
plt.axhline(y=0.67, color='black', linestyle='--', linewidth=0.9)  # Línea horizontal
plt.axvline(x=2.5, color='black', linestyle='--', linewidth=0.9)  # Línea vertical

# Añadir texto para las regiones
plt.text(0, 1.35, 'T-Region', fontsize=13, ha='center', va='bottom', color='black')
plt.text(0.75, -0.25, 'LT-Region', fontsize=13, ha='center', va='bottom', color='black')
plt.text(8, 1, 'ET-Region', fontsize=13, ha='center', va='bottom', color='black')
plt.text(8, -0.25, 'O-Region', fontsize=13, ha='center', va='bottom', color='black')

# Añadir la leyenda
plt.legend(loc='upper right', fontsize=12)
plt.tight_layout()

# Mostrar el gráfico
plt.show()
print(len(decals_no_zero['n_r'][mask_early]))

In [ ]:
x_valid = n_r[mask_late]
y_valid = g_r_decals[mask_late]

## Filtra valores válidos
#valid_mask = ~np.isnan(x_valid) & ~np.isnan(y_valid) & ~np.isinf(x_valid) & ~np.isinf(y_valid)
#x_valid = x_valid[valid_mask]
#y_valid = y_valid[valid_mask]

late_type_sfcg = np.sum([(y_valid <= 0.67) & (x_valid <= 2.5)])
transition_sfcg = np.sum([(y_valid > 0.67) & (x_valid <= 2.5)])
early_type_sfcg = np.sum([(y_valid >= 0.67) & (x_valid > 2.5)])
other_sfcg = np.sum([(y_valid < 0.67) & (x_valid > 2.5)])
print(late_type_sfcg/len(x_valid), transition_sfcg/len(x_valid), early_type_sfcg/len(x_valid), other_sfcg/len(x_valid))
print(len(x_valid), late_type_sfcg, transition_sfcg, early_type_sfcg, other_sfcg)

In [ ]:
H_0 = 67.4 #km/s/Mpc
c = 300000
sfr_decals['Distance (Mpc)'] = (sfr_decals['zmemb']*c)/H_0
sfr_decals['M_i'] = sfr_decals['m_i_corrected'] - 5*np.log10(sfr_decals['Distance (Mpc)']*1000000) + 5 + 1.592*sfr_decals['ebv']
sfr_decals['stellar_mass_phot_i'] = (1.15 + 0.7*(sfr_decals['m_g_corrected'] - sfr_decals['m_i_corrected']) - 0.4*sfr_decals['M_i'])
sfr_decals['log_sSFR'] = sfr_decals['logSFR_NUV'] - sfr_decals['stellar_mass_phot_i']

mascara = [
    isinstance(val1, float) and isinstance(val2, float) 
    for val1, val2 in zip(sfr_decals['logSFR_NUV'], sfr_decals['stellar_mass_phot_i'])
]

# Filtrar la tabla usando la máscara
sfr_decals = sfr_decals[mascara]
sfr_decals = sfr_decals[sfr_decals['log_sSFR'] > -50]
sfr_decals = sfr_decals[(sfr_decals['A'] != 100) & (sfr_decals['A'] > 0)]
len(sfr_decals)

In [ ]:
sfr_cs = Table.read('sfr_cs_k_corrected_asmlb.csv', format='csv')

# Crear una máscara para detectar valores válidos (no NaN y no Inf)
mascara = ~np.isnan(sfr_cs['logSFR_NUV']) & np.isfinite(sfr_cs['logSFR_NUV'])

# Aplicar la máscara a la tabla
sfr_cs = sfr_cs[mascara]

# Clasificación y creación de la columna 'tipo'
tipo = []
for gini, m20 in zip(sfr_cs['Gini'], sfr_cs['M20']):
    if gini < line_1(m20) and gini > line_2(m20): 
        tipo.append('Early')
    elif gini <= line_1(m20) and gini < line_2(m20): 
        tipo.append('Late')
    else:  
        tipo.append('Merger')

# Agregar la columna 'tipo' a la tabla original
sfr_cs['Tipo'] = Column(tipo, dtype='str')
H_0 = 67.4 #km/s/Mpc
c = 300000
sfr_cs['Distance (Mpc)'] = (sfr_cs['zmemb']*c)/H_0
sfr_cs['M_i'] = sfr_cs['m_i_corrected'] - 5*np.log10(sfr_cs['Distance (Mpc)']*1000000) + 5 + 1.592*sfr_cs['ebv']
sfr_cs['stellar_mass_phot_i'] = (1.15 + 0.7*(sfr_cs['m_g_corrected'] - sfr_cs['m_i_corrected']) - 0.4*sfr_cs['M_i'])
sfr_cs['log_sSFR'] = sfr_cs['logSFR_NUV'] - sfr_cs['stellar_mass_phot_i']

mascara = [
    isinstance(val1, float) and isinstance(val2, float) 
    for val1, val2 in zip(sfr_cs['logSFR_NUV'], sfr_cs['stellar_mass_phot_i'])
]

# Filtrar la tabla usando la máscara
sfr_cs = sfr_cs[mascara]
sfr_cs = sfr_cs[(sfr_cs['A'] != 100) & (sfr_cs['A'] > 0)]
len(sfr_cs)

In [ ]:
merger_galaxies = sfr_decals[sfr_decals['Tipo'] == 'Merger']
late_galaxies = sfr_decals[sfr_decals['Tipo'] == 'Late']
early_galaxies = sfr_decals[sfr_decals['Tipo'] == 'Early']

merger_galaxies_cs = sfr_cs[sfr_cs['Tipo'] == 'Merger']
late_galaxies_cs = sfr_cs[sfr_cs['Tipo'] == 'Late']
early_galaxies_cs = sfr_cs[sfr_cs['Tipo'] == 'Early']
print(len(merger_galaxies_cs), len(late_galaxies_cs), len(early_galaxies_cs))

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

sfr_mergers = merger_galaxies['logSFR_NUV']
sfr_late = late_galaxies['logSFR_NUV']
sfr_early = early_galaxies['logSFR_NUV']
# Crear bins
bin_edges = np.linspace(min(sfr_mergers.min(), sfr_late.min(), sfr_early.min()), 
                        max(sfr_mergers.max(), sfr_late.max(), sfr_early.max()), 
                        21)
# Ejemplo de uso para tus distribuciones
median_mergers, err_low_mergers, err_high_mergers = bootstrap_median_error(sfr_mergers)
median_late, err_low_late, err_high_late = bootstrap_median_error(sfr_late)
median_early, err_low_early, err_high_early = bootstrap_median_error(sfr_early)

# Imprimir resultados
print(f"Mergers Median: {median_mergers:.2f} (+{err_high_mergers:.2f} / -{err_low_mergers:.2f})")
print(f"Late Median: {median_late:.2f} (+{err_high_late:.2f} / -{err_low_late:.2f})")
print(f"Early Median: {median_early:.2f} (+{err_high_early:.2f} / -{err_low_early:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(sfr_mergers, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='darkgreen', label='Mergers', edgecolor='black', linewidth=0.3)
sns.histplot(sfr_late, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='cornflowerblue', label='Late', edgecolor='black', linewidth=0.3)
sns.histplot(sfr_early, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='indianred', label='Early', edgecolor='black', linewidth=0.3)

plt.axvline(median_mergers, color='darkgreen', linestyle='--', linewidth=3, label='Median Mergers')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers - err_low_mergers, x2=median_mergers + err_high_mergers, 
                  color='darkgreen', alpha=0.4)
plt.axvline(median_late, color='cornflowerblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late - err_low_late, x2=median_late + err_high_late, 
                  color='cornflowerblue', alpha=0.4)
plt.axvline(median_early, color='indianred', linestyle='--', linewidth=3, label='Median Early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early - err_low_early, x2=median_early + err_high_early, 
                  color='indianred', alpha=0.4)

# Etiquetas y título
plt.xlabel('log SFR $[M_{\odot}/yr]$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

sfr_mergers = merger_galaxies['logSFR_NUV']
sfr_mergers_cs = merger_galaxies_cs['logSFR_NUV']
# Crear bins
bin_edges = np.linspace(min(sfr_mergers.min(), sfr_mergers_cs.min()), 
                        max(sfr_mergers.max(), sfr_mergers_cs.max()), 
                        15)
# Ejemplo de uso para tus distribuciones
median_mergers, err_low_mergers, err_high_mergers = bootstrap_median_error(sfr_mergers)
median_mergers_cs, err_low_mergers_cs, err_high_mergers_cs = bootstrap_median_error(sfr_mergers_cs)

# Imprimir resultados
print(f"Mergers Median in SFCGs: {median_mergers:.2f} (+{err_high_mergers:.2f} / -{err_low_mergers:.2f})")
print(f"Mergers Median in the Field: {median_mergers_cs:.2f} (+{err_high_mergers_cs:.2f} / -{err_low_mergers_cs:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(sfr_mergers, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='darkgreen', label='Mergers', edgecolor='black', linewidth=0.3)
sns.histplot(sfr_mergers_cs, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='sandybrown', label='Late', edgecolor='black', linewidth=0.3)

plt.axvline(median_mergers, color='darkgreen', linestyle='--', linewidth=3, label='Median Mergers')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers - err_low_mergers, x2=median_mergers + err_high_mergers, 
                  color='darkgreen', alpha=0.4)
plt.axvline(median_mergers_cs, color='sandybrown', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers_cs - err_low_mergers_cs, x2=median_mergers_cs + err_high_mergers_cs, 
                  color='sandybrown', alpha=0.4)


# Etiquetas y título
plt.xlabel('log SFR $[M_{\odot}/yr]$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

sfr_late = late_galaxies['logSFR_NUV']
sfr_late_cs = late_galaxies_cs['logSFR_NUV']
# Crear bins
bin_edges = np.linspace(min(sfr_late.min(), sfr_late_cs.min()), 
                        max(sfr_late.max(), sfr_late_cs.max()), 
                        21)
# Ejemplo de uso para tus distribuciones
median_late, err_low_late, err_high_late = bootstrap_median_error(sfr_late)
median_late_cs, err_low_late_cs, err_high_late_cs = bootstrap_median_error(sfr_late_cs)

# Imprimir resultados
print(f"Late Median in SFCGs: {median_late:.2f} (+{err_high_late:.2f} / -{err_low_late:.2f})")
print(f"Late Median in the Field: {median_late_cs:.2f} (+{err_high_late_cs:.2f} / -{err_low_late_cs:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(sfr_late, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='cornflowerblue', label='Late', edgecolor='black', linewidth=0.3)
sns.histplot(sfr_late_cs, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='lightsteelblue', label='Late', edgecolor='black', linewidth=0.3)

plt.axvline(median_late, color='cornflowerblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late - err_low_late, x2=median_late + err_high_late, 
                  color='cornflowerblue', alpha=0.4)
plt.axvline(median_late_cs, color='lightsteelblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late_cs - err_low_late_cs, x2=median_late_cs + err_high_late_cs, 
                  color='lightsteelblue', alpha=0.4)


# Etiquetas y título
plt.xlabel('log SFR $[M_{\odot}/yr]$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

sfr_early = early_galaxies['logSFR_NUV']
sfr_early_cs = early_galaxies_cs['logSFR_NUV']
# Crear bins
bin_edges = np.linspace(min(sfr_early.min(), sfr_early_cs.min()), 
                        max(sfr_early.max(), sfr_early_cs.max()), 
                        21)
# Ejemplo de uso para tus distribuciones
median_early, err_low_early, err_high_early = bootstrap_median_error(sfr_early)
median_early_cs, err_low_early_cs, err_high_early_cs = bootstrap_median_error(sfr_early_cs)

# Imprimir resultados
print(f"early Median in SFCGs: {median_early:.2f} (+{err_high_early:.2f} / -{err_low_early:.2f})")
print(f"early Median in the Field: {median_early_cs:.2f} (+{err_high_early_cs:.2f} / -{err_low_early_cs:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(sfr_early, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='indianred', label='early', edgecolor='black', linewidth=0.3)
sns.histplot(sfr_early_cs, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='grey', label='early', edgecolor='black', linewidth=0.3)

plt.axvline(median_early, color='indianred', linestyle='--', linewidth=3, label='Median early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early - err_low_early, x2=median_early + err_high_early, 
                  color='indianred', alpha=0.4)
plt.axvline(median_early_cs, color='grey', linestyle='--', linewidth=3, label='Median early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early_cs - err_low_early_cs, x2=median_early_cs + err_high_early_cs, 
                  color='grey', alpha=0.4)


# Etiquetas y título
plt.xlabel('log SFR $[M_{\odot}/yr]$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

ssfr_mergers = merger_galaxies['log_sSFR']
ssfr_late = late_galaxies['log_sSFR']
ssfr_early = early_galaxies['log_sSFR']
# Crear bins
bin_edges = np.linspace(min(ssfr_mergers.min(), ssfr_late.min()),# ssfr_early.min()), 
                        max(ssfr_mergers.max(), ssfr_late.max()),# ssfr_early.max()), 
                        21)
# Ejemplo de uso para tus distribuciones
median_mergers, err_low_mergers, err_high_mergers = bootstrap_median_error(ssfr_mergers)
median_late, err_low_late, err_high_late = bootstrap_median_error(ssfr_late)
median_early, err_low_early, err_high_early = bootstrap_median_error(ssfr_early)

# Imprimir resultados
print(f"Mergers Median: {median_mergers:.2f} (+{err_high_mergers:.2f} / -{err_low_mergers:.2f})")
print(f"Late Median: {median_late:.2f} (+{err_high_late:.2f} / -{err_low_late:.2f})")
print(f"Early Median: {median_early:.2f} (+{err_high_early:.2f} / -{err_low_early:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(ssfr_mergers, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='darkgreen', label='Mergers', edgecolor='black', linewidth=0.3)
sns.histplot(ssfr_late, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='cornflowerblue', label='Late', edgecolor='black', linewidth=0.3)
sns.histplot(ssfr_early, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='indianred', label='Early', edgecolor='black', linewidth=0.3)

plt.axvline(median_mergers, color='darkgreen', linestyle='--', linewidth=3, label='Median Mergers')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers - err_low_mergers, x2=median_mergers + err_high_mergers, 
                  color='darkgreen', alpha=0.4)
plt.axvline(median_late, color='cornflowerblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late - err_low_late, x2=median_late + err_high_late, 
                  color='cornflowerblue', alpha=0.4)
plt.axvline(median_early, color='indianred', linestyle='--', linewidth=3, label='Median Early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early - err_low_early, x2=median_early + err_high_early, 
                  color='indianred', alpha=0.4)

# Etiquetas y título
plt.xlabel('log sSFR $[yr^{-1}]$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

ssfr_mergers_cs = merger_galaxies_cs['log_sSFR']
ssfr_late_cs = late_galaxies_cs['log_sSFR']
ssfr_early_cs = early_galaxies_cs['log_sSFR']
# Crear bins
bin_edges = np.linspace(min(ssfr_mergers_cs.min(), ssfr_late_cs.min()),# ssfr_early.min()), 
                        max(ssfr_mergers_cs.max(), ssfr_late_cs.max()),# ssfr_early.max()), 
                        21)
# Ejemplo de uso para tus distribuciones
median_mergers, err_low_mergers, err_high_mergers = bootstrap_median_error(ssfr_mergers_cs)
median_late, err_low_late, err_high_late = bootstrap_median_error(ssfr_late_cs)
median_early, err_low_early, err_high_early = bootstrap_median_error(ssfr_early_cs)

# Imprimir resultados
print(f"Mergers Median: {median_mergers:.2f} (+{err_high_mergers:.2f} / -{err_low_mergers:.2f})")
print(f"Late Median: {median_late:.2f} (+{err_high_late:.2f} / -{err_low_late:.2f})")
print(f"Early Median: {median_early:.2f} (+{err_high_early:.2f} / -{err_low_early:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(ssfr_mergers_cs, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='darkgreen', label='Mergers', edgecolor='black', linewidth=0.3)
sns.histplot(ssfr_late_cs, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='cornflowerblue', label='Late', edgecolor='black', linewidth=0.3)
sns.histplot(ssfr_early_cs, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='indianred', label='Early', edgecolor='black', linewidth=0.3)

plt.axvline(median_mergers, color='darkgreen', linestyle='--', linewidth=3, label='Median Mergers')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers - err_low_mergers, x2=median_mergers + err_high_mergers, 
                  color='darkgreen', alpha=0.4)
plt.axvline(median_late, color='cornflowerblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late - err_low_late, x2=median_late + err_high_late, 
                  color='cornflowerblue', alpha=0.4)
plt.axvline(median_early, color='indianred', linestyle='--', linewidth=3, label='Median Early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early - err_low_early, x2=median_early + err_high_early, 
                  color='indianred', alpha=0.4)

# Etiquetas y título
plt.xlabel('log sSFR $[yr^{-1}]$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
from matplotlib.lines import Line2D

# Ignorar los warnings
warnings.filterwarnings("ignore")

# Configurar el estilo de Seaborn
sns.set_theme(style="ticks")
sns.set_palette("pastel")
custom_palette = {'Merger': 'darkgreen', 'Late': 'cornflowerblue', 'Early' : 'indianred'}


sur = sfr_decals['Tipo']

# Lista de columnas que quieres graficar
x = sfr_decals['log_sSFR']
y = sfr_decals['n_r']
# Crear la figura
g = sns.JointGrid(data=sfr_decals, x=x, y=y, hue=sur, palette=custom_palette, height=6)
    
# Gráficos principales: KDE con efecto desvanecido
g.plot_joint(
    sns.kdeplot,
    fill=False,
    alpha=1,
    linewidth=0,  # Contornos suaves
    )
    
# Gráficos marginales: KDE relleno
g.plot_marginals(
    sns.kdeplot,
    fill=True,
    common_norm=False,
    alpha=0.4,  # Transparencia
    linewidth=2  # Sin bordes
)
# Limitar los ejes
g.ax_joint.set_xlim(-13, -5.5)
g.ax_joint.set_ylim(-1, 8)
# Añadir títulos
g.set_axis_labels("log sSFR($yr^{-1}$)", f"$n_r$", fontsize=15)

# Crear handles y labels para la leyenda
handles = [Line2D([0], [0], color=custom_palette[name], lw=3) for name in custom_palette]
labels = list(custom_palette.keys())

# Añadir la leyenda
g.ax_joint.legend(handles=handles, labels=labels, loc="upper left")

#g.fig.suptitle(f"Late Type Galaxies", fontsize=14)  # Título principal
g.fig.subplots_adjust(top=0.93)  # Ajustar para que el título no se superponga
# Mostrar el gráfico
plt.show()

In [ ]:
from matplotlib.lines import Line2D

# Ignorar los warnings
warnings.filterwarnings("ignore")

# Configurar el estilo de Seaborn
sns.set_theme(style="ticks")
sns.set_palette("pastel")
custom_palette = {'Merger': 'darkgreen', 'Late': 'cornflowerblue', 'Early' : 'indianred'}


sur = sfr_decals['Tipo']

# Lista de columnas que quieres graficar
x = sfr_decals['log_sSFR']
y = sfr_decals['A']
# Crear la figura
g = sns.JointGrid(data=sfr_decals, x=x, y=y, hue=sur, palette=custom_palette, height=6)
    
# Gráficos principales: KDE con efecto desvanecido
g.plot_joint(
    sns.kdeplot,
    fill=False,
    alpha=1,
    linewidth=0,  # Contornos suaves
    )
    
# Gráficos marginales: KDE relleno
g.plot_marginals(
    sns.kdeplot,
    fill=True,
    common_norm=False,
    alpha=0.4,  # Transparencia
    linewidth=2  # Sin bordes
)
# Limitar los ejes
g.ax_joint.set_xlim(-13, -6)
g.ax_joint.set_ylim(-0.1, 1)
# Añadir títulos
g.set_axis_labels("log sSFR($yr^{-1}$)", f"$A$", fontsize=15)

# Crear handles y labels para la leyenda
handles = [Line2D([0], [0], color=custom_palette[name], lw=3) for name in custom_palette]
labels = list(custom_palette.keys())

# Añadir la leyenda
g.ax_joint.legend(handles=handles, labels=labels, loc="upper left")

#g.fig.suptitle(f"Late Type Galaxies", fontsize=14)  # Título principal
g.fig.subplots_adjust(top=0.93)  # Ajustar para que el título no se superponga
# Mostrar el gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

assymetry_mergers = merger_galaxies['A']
assymetry_late = late_galaxies['A']
assymetry_early = early_galaxies['A']
# Crear bins
bin_edges = np.linspace(min(assymetry_mergers.min(), assymetry_late.min()),# ssfr_early.min()), 
                        max(assymetry_mergers.max(), assymetry_late.max()),# ssfr_early.max()), 
                        21)
# Ejemplo de uso para tus distribuciones
median_mergers, err_low_mergers, err_high_mergers = bootstrap_median_error(assymetry_mergers)
median_late, err_low_late, err_high_late = bootstrap_median_error(assymetry_late)
median_early, err_low_early, err_high_early = bootstrap_median_error(assymetry_early)

# Imprimir resultados
print(f"Mergers Median: {median_mergers:.2f} (+{err_high_mergers:.2f} / -{err_low_mergers:.2f})")
print(f"Late Median: {median_late:.2f} (+{err_high_late:.2f} / -{err_low_late:.2f})")
print(f"Early Median: {median_early:.2f} (+{err_high_early:.2f} / -{err_low_early:.2f})")


# Crear la figura
plt.figure(figsize=(10, 6))

# Histogramas
sns.histplot(assymetry_mergers, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='darkgreen', label='Mergers', edgecolor='black', linewidth=0.3)
sns.histplot(assymetry_late, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='cornflowerblue', label='Late', edgecolor='black', linewidth=0.3)
sns.histplot(assymetry_early, kde=True, stat='probability', alpha=0.5, bins=bin_edges, 
             color='indianred', label='Early', edgecolor='black', linewidth=0.3)

plt.axvline(median_mergers, color='darkgreen', linestyle='--', linewidth=3, label='Median Mergers')
plt.fill_betweenx(y=[0, 0.25], x1=median_mergers - err_low_mergers, x2=median_mergers + err_high_mergers, 
                  color='darkgreen', alpha=0.4)
plt.axvline(median_late, color='cornflowerblue', linestyle='--', linewidth=3, label='Median Late')
plt.fill_betweenx(y=[0, 0.25], x1=median_late - err_low_late, x2=median_late + err_high_late, 
                  color='cornflowerblue', alpha=0.4)
plt.axvline(median_early, color='indianred', linestyle='--', linewidth=3, label='Median Early')
plt.fill_betweenx(y=[0, 0.25], x1=median_early - err_low_early, x2=median_early + err_high_early, 
                  color='indianred', alpha=0.4)

# Etiquetas y título
plt.xlabel('$A$', fontsize=15)
plt.ylabel('Fraction of galaxies', fontsize=15)
plt.xticks(size=14)
plt.yticks(size=14)

# Leyenda
plt.legend(fontsize=12)

# Mostrar gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

# Configurar el estilo de Seaborn
sns.set_theme(style="ticks")
sns.set_palette("pastel")
custom_palette = {'Merger': 'darkgreen', 'Late': 'cornflowerblue', 'Early' : 'indianred'}


sur = sfr_decals['Tipo']

# Lista de columnas que quieres graficar
x = sfr_decals['stellar_mass_phot_i']
y = sfr_decals['logSFR_NUV']
# Crear la figura
g = sns.JointGrid(data=sfr_decals, x=x, y=y, hue=sur, palette=custom_palette, height=6)
    
# Gráficos principales: KDE con efecto desvanecido
g.plot_joint(
    sns.kdeplot,
    fill=False,
    alpha=1,
    linewidth=0,  # Contornos suaves
    )
    
# Gráficos marginales: KDE relleno
g.plot_marginals(
    sns.kdeplot,
    fill=True,
    common_norm=False,
    alpha=0.4,  # Transparencia
    linewidth=2  # Sin bordes
)
# Limitar los ejes
g.ax_joint.set_xlim(6, 12.5)
g.ax_joint.set_ylim(-2.3, 3)
# Añadir títulos
g.set_axis_labels("$log M_{*}[M_{\odot}$)", "$log SFR [M_{\odot}/yr]$", fontsize=15)

# Crear handles y labels para la leyenda
handles = [Line2D([0], [0], color=custom_palette[name], lw=3) for name in custom_palette]
labels = list(custom_palette.keys())

# Añadir la leyenda
g.ax_joint.legend(handles=handles, labels=labels, loc="upper left")

#g.fig.suptitle(f"Late Type Galaxies", fontsize=14)  # Título principal
g.fig.subplots_adjust(top=0.93)  # Ajustar para que el título no se superponga
# Mostrar el gráfico
plt.show()

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

# Configurar el estilo de Seaborn
sns.set_theme(style="ticks")
sns.set_palette("pastel")
custom_palette = {'Merger': 'darkgreen', 'Late': 'cornflowerblue', 'Early' : 'indianred'}


sur = sfr_cs['Tipo']

# Lista de columnas que quieres graficar
x = sfr_cs['log_sSFR']
y = sfr_cs['A']
# Crear la figura
g = sns.JointGrid(data=sfr_cs, x=x, y=y, hue=sur, palette=custom_palette, height=6)
    
# Gráficos principales: KDE con efecto desvanecido
g.plot_joint(
    sns.kdeplot,
    fill=False,
    alpha=1,
    linewidth=0,  # Contornos suaves
    )
    
# Gráficos marginales: KDE relleno
g.plot_marginals(
    sns.kdeplot,
    fill=True,
    common_norm=False,
    alpha=0.4,  # Transparencia
    linewidth=2  # Sin bordes
)
# Limitar los ejes
g.ax_joint.set_xlim(-13, -6)
g.ax_joint.set_ylim(-0.1, 0.7)
# Añadir títulos
g.set_axis_labels("log sSFR($yr^{-1}$)", f"$A$", fontsize=15)

# Crear handles y labels para la leyenda
handles = [Line2D([0], [0], color=custom_palette[name], lw=3) for name in custom_palette]
labels = list(custom_palette.keys())

# Añadir la leyenda
g.ax_joint.legend(handles=handles, labels=labels, loc="upper left")

#g.fig.suptitle(f"Late Type Galaxies", fontsize=14)  # Título principal
g.fig.subplots_adjust(top=0.93)  # Ajustar para que el título no se superponga
# Mostrar el gráfico
plt.show()

In [ ]:
type_column = Column(['SFCG Galaxy'] * len(sfr_decals), name='Nature')
sfr_decals.add_column(type_column)

type_column_cs = Column(['Field Galaxy'] * len(sfr_cs), name='Nature')
sfr_cs.add_column(type_column_cs)

sfr_decals_columns = sfr_decals['Gini', 'M20', 'A', 'C', 'log_sSFR', 'logSFR_NUV', 'stellar_mass_phot_i', 'Tipo', 'Nature']
sfr_cs_columns = sfr_cs['Gini', 'M20', 'A', 'C', 'log_sSFR', 'logSFR_NUV', 'stellar_mass_phot_i', 'Tipo', 'Nature']

cg_cs = vstack([sfr_decals_columns, sfr_cs_columns])

In [ ]:
# Ignorar los warnings
warnings.filterwarnings("ignore")

# Configurar el estilo de Seaborn
sns.set_theme(style="ticks")
sns.set_palette("pastel")
custom_palette = {'SFCG Galaxy': 'darkviolet', 'Field Galaxy': 'teal'}

data = cg_cs[cg_cs['Tipo'] == 'Merger']
sur = data['Nature']
# Lista de columnas que quieres graficar
x = data['log_sSFR']
y = data['A']
# Crear la figura
g = sns.JointGrid(data=data, x=x, y=y, hue=sur, palette=custom_palette, height=6)
    
# Gráficos principales: KDE con efecto desvanecido
g.plot_joint(
    sns.kdeplot,
    fill=False,
    alpha=1,
    linewidth=0,  # Contornos suaves
    )
    
# Gráficos marginales: KDE relleno
g.plot_marginals(
    sns.kdeplot,
    fill=True,
    common_norm=False,
    alpha=0.4,  # Transparencia
    linewidth=2  # Sin bordes
)
# Limitar los ejes
g.ax_joint.set_xlim(-13, -5.7)
g.ax_joint.set_ylim(-0.2, 1)
# Añadir títulos
g.set_axis_labels("log sSFR($yr^{-1}$)", f"$A$", fontsize=15)

# Crear handles y labels para la leyenda
handles = [Line2D([0], [0], color=custom_palette[name], lw=3) for name in custom_palette]
labels = list(custom_palette.keys())

# Añadir la leyenda
g.ax_joint.legend(handles=handles, labels=labels, loc="upper left")

#g.fig.suptitle(f"Late Type Galaxies", fontsize=14)  # Título principal
g.fig.subplots_adjust(top=0.93)  # Ajustar para que el título no se superponga
# Mostrar el gráfico
plt.show()